# Text to Code Explanation

In [3]:
import sys
!{sys.executable} -m pip install --upgrade --user google-cloud-aiplatform google-cloud-storage 'google-cloud-bigquery[pandas]'

  Using cached google_cloud_aiplatform-1.36.4-py2.py3-none-any.whl.metadata (27 kB)
Using cached google_cloud_aiplatform-1.36.4-py2.py3-none-any.whl (3.3 MB)


In [4]:
# restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

**Set Environment Variables**

In [10]:
# get project ID
PROJECT_ID = ! gcloud config get project
#PROJECT_ID = 'qwiklabs-asl-04-06351f77b64f'
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1"
if PROJECT_ID == "(unset)":
    print(f"Please set the project ID manually below")

In [11]:
# define project information
if PROJECT_ID == "(unset)":
    PROJECT_ID = "qwiklabs-asl-04-06351f77b64f" # @param {type:"string"}

# generate an unique id for this session
from datetime import datetime
UID = datetime.now().strftime("%m%d%H%M")

In [12]:
#! gcloud services enable compute.googleapis.com aiplatform.googleapis.com storage.googleapis.com bigquery.googleapis.com --project {PROJECT_ID}

**Data Preparation**

In [18]:
import pandas as pd
from google.cloud import bigquery

bq_client = bigquery.Client(project=PROJECT_ID)
QUERY_TEMPLATE = """
        SELECT *
        FROM `qwiklabs-asl-04-e7ad74c2e059".rax_dataset_metadata.table_metadata`
        UNION ALL
        SELECT *
        FROM `qwiklabs-asl-04-e7ad74c2e059".rax_dataset_metadata.procedure_metadata`
        """
query = QUERY_TEMPLATE.format()
query_job = bq_client.query(QUERY_TEMPLATE)
rows = query_job.result()
df = rows.to_dataframe()

# examine the data
df.head()

,table_schema,table_name,ddl
0,rax_bq_metadata,account_ran,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...
1,rax_bq_metadata,email_apps_tax_detail_report,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...
2,rax_bq_metadata,dctm_comments_assignments_and_work,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...
3,rax_bq_metadata,wismo,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...
4,rax_bq_metadata,bt_pipeline_waterfall_week,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...


**Call the API to generate embeddings**

We will use the ddl column and generate embedding for it with Embeddings for Text API

In [ ]:
# init the vertexai package
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
# Load the text embeddings model
from vertexai.preview.language_models import TextEmbeddingModel
model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")

Printing for loop to understand tqdm

In [22]:
import tqdm
texts = [1,2,3,4,5,9,7,8,9,10,11,12,13,14,21,16]
for i in tqdm.tqdm(range(0, len(texts), 5)):
    print(texts[i:i + 5])

100%|██████████| 4/4 [00:00<00:00, 25191.02it/s]

[1, 2, 3, 4, 5]
[9, 7, 8, 9, 10]
[11, 12, 13, 14, 21]
[16]


In [23]:
import time
import tqdm # to show a progress bar

# get embeddings for a list of texts
BATCH_SIZE = 5
def get_embeddings_wrapper(texts):
    embs = [] # creating an empty list to store embeddings
    for i in tqdm.tqdm(range(0, len(texts), BATCH_SIZE)): # looping from 0 till the length of texts with a step of BATCH_SIZE = 5 (0, 5, 10, 15...)
        time.sleep(1) # to avoid the quota error
        result = model.get_embeddings(texts[i:i + BATCH_SIZE]) # get embeddings for a batch of texts defined by i and i + BATCH_SIZE (0:0+5, 5:5+5, 10:10+5...)
        embs = embs + [e.values for e in result] # storing the values returned by the get_embeddings function in the embs empty list
    return embs

In [ ]:
# get embeddings for the ddl column and add them as "embedding" column in the dataframe
df = df.assign(embedding=get_embeddings_wrapper(list(df.ddl)))
df.head()

**Calculating Similarities - how these embeddings are organized in the embedding space with their meanings**

Calculate similarity between two embeddings by using one of the popular metrics like L2 distance, cosine similarity, inner product

In [25]:
import random
import numpy as np

# pick one of them as a key code
key = random.randint(0, len(df))

# calculate dot product between the key and other ddl
embs = np.array(df.embedding.to_list())
similarities = np.dot(embs[key], embs.T)

# print similarities for the first 5 codes
similarities[:5]

array([0.85769618, 0.83066232, 0.9086    , 0.91825791, 0.87504539])

In [27]:
# print the codes
print(f"Key code: {df.ddl[key]}\n")

# sort and print the code by similarities
sorted_code = sorted(zip(df.ddl, similarities), key=lambda x: x[1], reverse=True)[:20]
for i, (code, similarity) in enumerate(sorted_code):
    print(f"{similarity:.4f} {code}")

Key code: CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax_bq_metadata.email_apps_tax_detail_report`
(
  account_number STRING,
  customer_name STRING,
  address STRING,
  city STRING,
  state STRING,
  zip STRING,
  country STRING,
  invoice_number STRING,
  invoiceyearmonth STRING,
  created_t DATETIME,
  invoice_date STRING,
  invoice_amount FLOAT64,
  invoice_balance FLOAT64,
  currency STRING
)
PARTITION BY DATETIME_TRUNC(created_t, MONTH)
CLUSTER BY country, customer_name, account_number, invoice_number
OPTIONS(
  require_partition_filter=true
);

1.0000 CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax_bq_metadata.email_apps_tax_detail_report`
(
  account_number STRING,
  customer_name STRING,
  address STRING,
  city STRING,
  state STRING,
  zip STRING,
  country STRING,
  invoice_number STRING,
  invoiceyearmonth STRING,
  created_t DATETIME,
  invoice_date STRING,
  invoice_amount FLOAT64,
  invoice_balance FLOAT64,
  currency STRING
)
PARTITION BY DATETIME_TRUNC(created_t, MONTH

Creating an index column named "id"

In [27]:
df['id']= df.index

In [28]:
df

,table_schema,table_name,ddl,id
0,rax_bq_metadata,account_ran,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,0
1,rax_bq_metadata,email_apps_tax_detail_report,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,1
2,rax_bq_metadata,dctm_comments_assignments_and_work,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,2
3,rax_bq_metadata,wismo,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,3
4,rax_bq_metadata,bt_pipeline_waterfall_week,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,4
...,...,...,...,...
172,rax_bq_metadata,rev_mat_opps_detail,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,172
173,rax_bq_metadata,retention_rate_ib,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,173
174,rax_bq_metadata,vw_alert_logic,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,174
175,rax_bq_metadata,device_availability,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,175


**Save the embeddings in a JSON file**

In [34]:
# save id and embedding as a json file
jsonl_string = df[['id', 'embedding']].to_json(orient = 'records', lines = True)
with open('codeexplanation.json', 'w') as f:
    f.write(jsonl_string)

# show the first few lines of the json file
! head -n 3 codeexplanation.json

{"id":0,"embedding":[-0.04515706,-0.050130114,-0.0235803258,0.0279327407,0.0156944953,-0.0556662232,0.0481634364,0.0071523716,-0.0035243982,0.0208040178,-0.0236611869,-0.0038096062,0.0067580477,0.0349338874,0.0257413834,-0.0190424006,-0.0004454835,0.0052142483,0.0196017455,0.0175804514,-0.0364599153,-0.0447966419,0.0103281904,0.0130845457,-0.0111046173,-0.0844000727,0.002823195,-0.0309756454,-0.0660549551,0.0337891951,-0.0487467349,0.0079384409,-0.0629910007,-0.0040504043,0.020544529,0.0205559842,-0.0008966839,0.0723990574,-0.0308351479,0.0575400926,0.0180797223,-0.0235445909,-0.0206735581,0.0148384236,-0.0301174279,-0.0405051783,0.0005752286,0.0343372114,-0.0058549936,-0.0784126893,-0.0636729747,-0.0408810601,-0.0238575526,0.0226222556,0.050668709,0.0151783209,0.0001675196,0.0469796918,0.001648635,-0.0132559184,0.0106282569,-0.0456846356,0.0127996132,-0.0125993527,0.0058746799,-0.0025804883,0.0198072549,0.0347900912,-0.0384252481,-0.022840023,0.0428690501,0.012363865,-0.017233571,-0.0

**Copy the file to a Cloud Storage bucket**

In [35]:
BUCKET_URI = f"gs://{PROJECT_ID}-embvs-codeexplanation"
! gsutil mb -l $LOCATION -p {PROJECT_ID} {BUCKET_URI}
! gsutil cp codeexplanation.json {BUCKET_URI}

Creating gs://qwiklabs-asl-04-06351f77b64f-embvs-codeexplanation/...
Copying file://codeexplanation.json [Content-Type=application/json]...
/ [1 files][915.1 KiB/915.1 KiB]                                                
Operation completed over 1 objects/915.1 KiB.                                    


In [29]:
df.head()

,table_schema,table_name,ddl,id
0,rax_bq_metadata,account_ran,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,0
1,rax_bq_metadata,email_apps_tax_detail_report,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,1
2,rax_bq_metadata,dctm_comments_assignments_and_work,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,2
3,rax_bq_metadata,wismo,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,3
4,rax_bq_metadata,bt_pipeline_waterfall_week,CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax...,4


In [ ]:
import os
BUCKET_URI = f"gs://{PROJECT_ID}-embvs-codeexplanation"
DOCUMENT_URI = os.path.join(BUCKET_URI, "documents")

for index, row in df.iterrows():
    path = f"./documents/{row.id}.txt"
    f = open(path, mode="w")
    f.write(row.ddl)
    gcspath = f"{DOCUMENT_URI}/{row.id}"
    !gsutil cp {path} {gcspath}

**Create Index**

In [36]:
# init the aiplatform package
from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

Create Vector Search with create_tree_ah_index function

In [37]:
# create index
my_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name = f"embvs-codeexplanation-index-{UID}",
    contents_delta_uri = BUCKET_URI,
    dimensions = 768,
    approximate_neighbors_count = 20,
    distance_measure_type = "DOT_PRODUCT_DISTANCE",
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/386130433895/locations/us-central1/indexes/3609709868128141312/operations/8343962227274940416
MatchingEngineIndex created. Resource name: projects/386130433895/locations/us-central1/indexes/3609709868128141312
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/386130433895/locations/us-central1/indexes/3609709868128141312')


**Create Index Endpoint and deploy the Index**

In [39]:
# create IndexEndpoint
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
  display_name = f"embvs-codeexplanation-index-endpoint-{UID}",
  public_endpoint_enabled = True,
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/386130433895/locations/us-central1/indexEndpoints/8823752336716333056/operations/5038320100784996352
MatchingEngineIndexEndpoint created. Resource name: projects/386130433895/locations/us-central1/indexEndpoints/8823752336716333056
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/386130433895/locations/us-central1/indexEndpoints/8823752336716333056')


In [55]:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/386130433895/locations/us-central1/indexEndpoints/8823752336716333056')
index_endpoint

resource name: projects/386130433895/locations/us-central1/indexEndpoints/8823752336716333056

With the Index Endpoint, deploy the Index by specifying an unique deployed index ID

In [40]:
DEPLOYED_INDEX_ID = f"embvs_codeexplanation_deployed_{UID}"

In [41]:
# deploy the Index to the Index Endpoint
my_index_endpoint.deploy_index(
  index = my_index, deployed_index_id = DEPLOYED_INDEX_ID
)

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/386130433895/locations/us-central1/indexEndpoints/8823752336716333056
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/386130433895/locations/us-central1/indexEndpoints/8823752336716333056/operations/8905786280789409792
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/386130433895/locations/us-central1/indexEndpoints/8823752336716333056


resource name: projects/386130433895/locations/us-central1/indexEndpoints/8823752336716333056

**Testing**

In [49]:
test_embeddings = get_embeddings_wrapper(["What is Call Metrics?"])

100%|██████████| 1/1 [00:01<00:00,  1.10s/it]


In [50]:
# Test query
response = my_index_endpoint.find_neighbors(
  deployed_index_id = DEPLOYED_INDEX_ID,
  queries = test_embeddings,
  num_neighbors = 5,
)

# show the result
import numpy as np
for idx, neighbor in enumerate(response[0]):
    id = np.int64(neighbor.id)
    similar = df.query("id == @id", engine = "python")
    print(f"{neighbor.distance:.4f} {similar.ddl.values[0]}")

0.6291 CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax_bq_metadata.customer_call_experience_finding_right_queue_attributes`
(
  frq_experience_contact_id STRING,
  finding_right_queue STRING,
  total_attempts INT64,
  same_queue_contact STRING,
  total_handled_contact_ids INT64,
  sum_queue_duration INT64,
  sum_agent_interaction_duration INT64
)
CLUSTER BY finding_right_queue;
0.6182 CREATE TABLE `qwiklabs-asl-04-e7ad74c2e059.rax_bq_metadata.call_availability_metrics_agent_metrics`
(
  _tmk_sso STRING,
  racker_start_time_key STRING,
  start_time_key_cst INT64,
  start_minute INT64,
  agent_sso STRING,
  start_datetime_utc DATETIME,
  start_hour_utc INT64,
  start_hour_minute_utc TIME,
  start_datetime_cst DATETIME,
  start_hour_cst INT64,
  start_hour_minute_cst TIME,
  end_datetime_cst DATETIME,
  agent_time_key STRING,
  routing_profile STRING,
  routing_profile_group STRING,
  agent_answer_rate STRING,
  occupancy STRING,
  contacts_agent_hung_up_first INT64,
  contacts_handled INT

In [45]:
import sys
!{sys.executable} -m pip install gradio